In [1]:
from BERT import BERT
from BERT import mlm_custom_loss, nsp_custom_loss
import numpy as np
import tensorflow as tf

DISERT_DATA_PATH = r'C:\Users\yunus\Desktop\DisERT Data'
CHECKPOINT_PATH = r'C:\Users\yunus\Desktop\Checkpoints\DisERT'
sCheckPointFilePath = f'{CHECKPOINT_PATH}\model'

X_MLM = np.load(f'{DISERT_DATA_PATH}\X_MLM.npy')
Y_MLM = np.load(f'{DISERT_DATA_PATH}\Y_MLM.npy')
X_NSP = np.load(f'{DISERT_DATA_PATH}\X_NSP.npy')
Y_NSP = np.load(f'{DISERT_DATA_PATH}\Y_NSP.npy')

X_MLM = X_MLM[:4000]
X_NSP = X_NSP[:4000]
Y_MLM = Y_MLM[:4000]
Y_NSP = Y_NSP[:4000]

In [2]:
oDisERT = BERT(
    mlm_input_shape = tuple([10, 129]), 
    nsp_input_shape = tuple([1, 129]),
    nr_of_encoder_blocks = 4,
    attention_key_dims = 32,
    attention_nr_of_heads = 2,
    attention_dense_dims = 128,
    dropout_rate = 0.0
)

oDisERT.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = [mlm_custom_loss, nsp_custom_loss]
)

oDisERT.fit(
    x = [X_MLM, X_NSP], 
    y = [Y_MLM, Y_NSP], 
    batch_size= 512,
    epochs=2,
    verbose=1
)

tf.keras.models.save_model(oDisERT, sCheckPointFilePath)

Epoch 1/2
8/8 [==============================] - 10s 288ms/step - loss: 1.2680 - output_1_loss: 0.0495 - output_2_loss: 1.2184
Epoch 2/2
8/8 [==============================] - 2s 277ms/step - loss: 0.7648 - output_1_loss: 0.0448 - output_2_loss: 0.7200


INFO:tensorflow:Assets written to: C:\Users\yunus\Desktop\Checkpoints\DisERT\model\assets


INFO:tensorflow:Assets written to: C:\Users\yunus\Desktop\Checkpoints\DisERT\model\assets


In [3]:
# oDisERT = tf.keras.models.load_model(
#     sCheckPointFilePath, 
#     custom_objects = {
#         'BERT':BERT,
#         # 'PositionEncoder':PositionEncoder,
#         # 'TransformerEncoder': TransformerEncoder,
#         'mlm_custom_loss':mlm_custom_loss,
#         'nsp_custom_loss':nsp_custom_loss
#     }
# )

# oDisERT.fit(
#     x = [X_MLM, X_NSP], 
#     y = [Y_MLM, Y_NSP], 
#     batch_size= 512,
#     epochs=5,
#     verbose=1,
#     # callbacks = [oDisERTCheckPoint, oCsvLogger, oLearningRateReducer, oEarlyStopper, oThresholdStopper]
# )

# tf.keras.models.save_model(oDisERT, sCheckPointFilePath)